In [1]:
import numpy as np

from model.PerceptronClassifier import PerceptronClassifier
from DataWork import mnist
from Trainer import split_reminder
from sklearn.metrics import classification_report
from pathlib import Path

In [2]:
perceptron = PerceptronClassifier.load_model(Path('EndPoint/1.pkl'))

In [3]:
layers = perceptron._layers
layers

In [4]:
backbone_layers = layers[0:4]
backbone_layers

In [5]:
emb_perceptron = PerceptronClassifier(backbone_layers)

In [6]:
(X_train, y_train, X_test, y_test) = mnist()
y_train = y_train % 2
y_test = y_test % 2

In [8]:
def predict(X, model):
    batched_X = split_reminder(X, 1)
    res = []
    for x in batched_X:
        out = model.forward(x)
        res.append(out)
    return np.stack(res).squeeze()


train_embeddings = predict(X_train, emb_perceptron)
test_embeddings = predict(X_test, emb_perceptron)

In [9]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=500)
fitted_log_reg = log_reg.fit(train_embeddings, y_train)

In [10]:
test_predicts = fitted_log_reg.predict(test_embeddings)

In [11]:
print(classification_report(y_true=y_test, y_pred=test_predicts))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93      4926
           1       0.93      0.94      0.94      5074

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000

